---
title: "Trade Envy Report"
date: last-modified
engine: jupyter
execute:
  echo: false
  enabled: true
---

In [1]:
import plotly.graph_objects as go
import psycopg2
import pandas as pd
from cow_amm_trade_envy.models import Pools
import os
from dotenv import load_dotenv

load_dotenv()

DB_PARAMS = {
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
}


def fetch_data_from_postgres(query: str, params: tuple = ()):
    """Fetch data from a Postgres database."""
    with psycopg2.connect(**DB_PARAMS) as conn:
        with conn.cursor() as cursor:
            cursor.execute(query, params)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
    return pd.DataFrame(data, columns=columns)


def analyze_surplus(network):
    table_name = f"{network}_envy"
    settlement_name = f"{network}_settle"
    query = (
        f"SELECT * FROM trade_envy.{table_name} te left join "
        f"trade_envy.{settlement_name} s on te.call_tx_hash = s.call_tx_hash"
    )

    df = fetch_data_from_postgres(query)
    negative_values = df["trade_envy"] < 0

    n_notna = df["pool"].notna().sum()

    print(f"{negative_values.sum()}/{n_notna} trades with negative envy")
    df = df[~negative_values]

    df = df[~df["pool"].isna()]
    df = df.drop(columns=["trade_index"])

    # Group by pool and calculate unused and used surplus
    grouped = df.groupby("pool").apply(
        lambda g: {
            "unused_surplus": g.loc[
                [not used for used in g["is_used"]], "trade_envy"
            ].sum(),
            "used_surplus": g.loc[g["is_used"], "trade_envy"].sum(),
        },
        include_groups=False,
    )

    results = grouped.reset_index().rename(columns={0: "surplus"})
    results["unused_surplus"] = results["surplus"].apply(lambda x: x["unused_surplus"])
    results["used_surplus"] = results["surplus"].apply(lambda x: x["used_surplus"])
    notna_pools = [not pd.isna(pool) for pool in results["pool"]]
    results.loc[notna_pools, "pool"] = results.loc[notna_pools, "pool"].apply(
        lambda x: Pools().get_name_from_address(x)
    )
    results = results.sort_values(by=["unused_surplus"], ascending=False)

    # also group by solver
    grouped_solver = df.groupby("solver").apply(
        lambda g: {
            "unused_surplus": g.loc[
                [not used for used in g["is_used"]], "trade_envy"
            ].sum(),
            "used_surplus": g.loc[df["is_used"], "trade_envy"].sum(),
        },
        include_groups=False,
    )

    results_solver = grouped_solver.reset_index().rename(columns={0: "surplus"})
    results_solver["unused_surplus"] = results_solver["surplus"].apply(
        lambda x: x["unused_surplus"]
    )
    results_solver["used_surplus"] = results_solver["surplus"].apply(
        lambda x: x["used_surplus"]
    )
    results_solver["solver"] = results_solver["solver"].str[:12]
    # sort
    results_solver = results_solver.sort_values(by=["unused_surplus"], ascending=False)

    return results, results_solver


# Analyze surplus for a given network
network = "ethereum"
surplus_data, surplus_per_solver = analyze_surplus(network)

# Prepare data for the stacked bar plot
pools = surplus_data["pool"]
unused_surplus = surplus_data["unused_surplus"]
used_surplus = surplus_data["used_surplus"]


# Additional function to plot surplus data
def plot_surplus(data, x_col, y1_col, y2_col, title, xaxis_title, yaxis_title):
    fig = go.Figure()

    # Add unused surplus (bottom of the stack)
    fig.add_trace(
        go.Bar(
            name="Unused Surplus",
            x=data[x_col],
            y=data[y1_col],
            marker_color="green",
        )
    )

    # Add used surplus (top of the stack)
    fig.add_trace(
        go.Bar(
            name="Used Surplus",
            x=data[x_col],
            y=data[y2_col],
            marker_color="orange",
        )
    )

    # Update layout for better visualization
    fig.update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        template="plotly",
        barmode="stack",  # Make the bars stacked
    )

    return fig


# Plot surplus per pool
pools_fig = plot_surplus(
    surplus_data,
    x_col="pool",
    y1_col="unused_surplus",
    y2_col="used_surplus",
    title="Comparison of Surplus Usage by Pool",
    xaxis_title="Pools",
    yaxis_title="Surplus Amount",
)
pools_fig.show()

# Prepare data for surplus per solver
solvers = surplus_per_solver["solver"]
unused_surplus_solver = surplus_per_solver["unused_surplus"]
used_surplus_solver = surplus_per_solver["used_surplus"]

# Plot surplus per solver
solvers_fig = plot_surplus(
    surplus_per_solver,
    x_col="solver",
    y1_col="unused_surplus",
    y2_col="used_surplus",
    title="Comparison of Surplus Usage by Solver",
    xaxis_title="Solvers",
    yaxis_title="Surplus Amount",
)
solvers_fig.show()

676/766 trades with negative envy
